In [52]:
! pip install mysql-connector-python

In [53]:
import mysql.connector
import pandas as pd
import mysql.connector


In [54]:
conn=mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="daproject2"
)

In [55]:
conn.is_connected()

True

In [56]:
#we will create an function out it

def connect_db():
    return mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="daproject2")

In [57]:
connect_db().is_connected()

True

In [58]:
db=connect_db()
cursor=db.cursor()

In [59]:
"select count(*) as total_soppliers from suppliers"

'select count(*) as total_soppliers from suppliers'

In [60]:
cursor.execute("select count(*) as total_suppliers from suppliers")

In [61]:
row=cursor.fetchone()


In [62]:
cursor.execute("SHOW TABLES")
print(cursor.fetchall())

[('products',), ('reorders',), ('shipments',), ('stock_entries',), ('suppliers',)]


In [63]:
# Re-fetch the suppliers count
cursor.execute("select count(*) as total_suppliers from suppliers")
row = cursor.fetchone()
print("Total suppliers:", row[0])

Total suppliers: 51


In [64]:
queries = {
"Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

"Total Products": "SELECT COUNT(*) AS count FROM products",

"Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

"Total Sale Value (Last 3 Months)": """
SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Sale'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Total Restock Value (Last 3 Months)": """
SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
FROM stock_entries se
JOIN products p ON se.product_id = p.product_id
WHERE se.change_type = 'Restock'
AND se.entry_date >= (
SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
""",

"Below Reorder & No Pending Reorders": """
SELECT COUNT(*) AS below_reorder
FROM products p
WHERE p.stock_quantity < p.reorder_level
AND p.product_id NOT IN (
SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
"""
}

In [65]:
result={}
for label,query in queries.items():
    cursor.execute(query)
    row=cursor.fetchone()
    result[label]=row[0]

In [66]:
result

{'Total Suppliers': 51,
 'Total Products': 201,
 'Total Categories Dealing': 6,
 'Total Sale Value (Last 3 Months)': Decimal('1529237.11'),
 'Total Restock Value (Last 3 Months)': Decimal('6690663.49'),
 'Below Reorder & No Pending Reorders': 14}

In [ ]:

## Now lets create function out of it 

def get_basic_info(cursor):
    """
    Retrieve summary inventory and supply chain metrics.

    Args:
        cursor (mysql.connector.cursor.MySQLCursorDict): Cursor object with dictionary=True.

    Returns:
        dict: Dictionary of metric labels and their values.
    """

    queries = {
        "Total Suppliers": "SELECT COUNT(*) AS count FROM suppliers",

        "Total Products": "SELECT COUNT(*) AS count FROM products",

        "Total Categories Dealing": "SELECT COUNT(DISTINCT category) AS count FROM products",

        "Total Sale Value (Last 3 Months)": """
            SELECT ROUND(SUM(ABS(se.change_quantity) * p.price), 2) AS total_sale
            FROM stock_entries se
            JOIN products p ON se.product_id = p.product_id
            WHERE se.change_type = 'Sale'
              AND se.entry_date >= (
                  SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Total Restock Value (Last 3 Months)": """
            SELECT ROUND(SUM(se.change_quantity * p.price), 2) AS total_restock
            FROM stock_entries se
            JOIN products p ON se.product_id = p.product_id
            WHERE se.change_type = 'Restock'
              AND se.entry_date >= (
                  SELECT DATE_SUB(MAX(entry_date), INTERVAL 3 MONTH) FROM stock_entries)
        """,

        "Below Reorder & No Pending Reorders": """
            SELECT COUNT(*) AS below_reorder
            FROM products p
            WHERE p.stock_quantity < p.reorder_level
              AND p.product_id NOT IN (
                  SELECT DISTINCT product_id FROM reorders WHERE status = 'Pending')
        """
    }

    results = {}
    for label, query in queries.items():
        cursor.execute(query)
        row = cursor.fetchone()
        # Since row is a dictionary, extract the single value by getting the first value in dict.values()
        results[label] = list(row.values())[0]

    return results



